In [1]:
import pandas as pd 
import os
from tqdm import tqdm
import tifffile
import numpy as np
import glob
import skimage.util

In [2]:
def download_images(metadata, folder_name_map):
    # aws s3 ls s3://cellpainting-gallery/cpg0000-jump-pilot/source_4/images/2020_11_04_CPJUMP1/images/BR00116991__2020-11-05T19_51_35-Measurement1/Images/
    aws_s3_prefix = f"aws s3 sync"
    sign_request = "--no-sign-request"
    folder = "cellpainting-gallery"
    accession_number = "cpg0000-jump-pilot"
    source = "source_4"

    metadata = (
        metadata.assign(row=lambda x: x.apply(lambda y: y.well_position[0:1], axis=1))
        .assign(col=lambda x: x.apply(lambda y: y.well_position[1:], axis=1))
        .replace(
            {
                "row": {
                    "A": "01",
                    "B": "02",
                    "C": "03",
                    "D": "04",
                    "E": "05",
                    "F": "06",
                    "G": "07",
                    "H": "08",
                    "I": "09",
                    "J": "10",
                    "K": "11",
                    "L": "12",
                    "M": "13",
                    "N": "14",
                    "O": "15",
                    "P": "16",
                }
            }
        )
    )

    plane = f"p01"
    channels = ["ch1", "ch2", "ch3", "ch4", "ch5"]

    for row in tqdm(metadata.itertuples(), total=metadata.shape[0]):
        batch = row.Batch
        perturbation = row.Perturbation
        plate = folder_name_map.query(
            "Assay_Plate_Barcode==@row.Assay_Plate_Barcode"
        ).folder_name.values[0]
        row_name = f"r{row.row}"
        col_name = f"c{row.col}"
        if row.Number_of_images > 30000: # hack for identifying wells with 16 fields of view.
            field = f"f08"
        else:
            field = f"f05"
        cell_type = row.Cell_type
        if perturbation == "compound":
            output = row.pert_iname
        else:
            output = row.gene

        if not os.path.isdir(f"figures/BRD4/{cell_type}/{perturbation}/{output}/"):
            os.mkdir(f"figures/BRD4/{cell_type}/{perturbation}/{output}/")
        
            for channel in channels:
                cmd = f'{aws_s3_prefix} {sign_request} s3://{folder}/{accession_number}/{source}/images/{batch}/images/{plate}/Images/ figures/BRD4/{cell_type}/{perturbation}/{output}/ --exclude "*" --include "{row_name}{col_name}{field}{plane}-{channel}*.tiff"'
                os.system(cmd)


Process metadata

In [3]:
batch = "2020_11_04_CPJUMP1"

experiment_df = (
    pd.read_csv("output/experiment-metadata.tsv", sep="\t")
    .query("Batch==@batch")
    .query("Density==100")
    .query('Antibiotics=="absent"')
    .query(
        "Time==48 or Time == 96 or Time==144"
    )  # Keep only the longer time points
)

experiment_df.drop(
    experiment_df[
        (experiment_df.Perturbation == "compound") & (experiment_df.Cell_line == "Cas9")
    ].index,
    inplace=True,
)

experiment_df.drop(
    experiment_df[
        (experiment_df.Perturbation == "crispr") & (experiment_df.Time == 96)
    ].index,
    inplace=True,
)

experiment_df.drop(
    experiment_df[
        (experiment_df.Perturbation == "orf") & (experiment_df.Time == 48)
    ].index,
    inplace=True,
)

# Read platemaps

compound_platemap = pd.read_csv(
    "../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_compound_platemap.txt",
    sep="\t",
    usecols=["well_position", "broad_sample"],
).assign(Plate_Map_Name="JUMP-Target-1_compound_platemap")

crispr_platemap = pd.read_csv(
    "../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_crispr_platemap.txt",
    sep="\t",
).assign(Plate_Map_Name="JUMP-Target-1_crispr_platemap")

orf_platemap = pd.read_csv(
    "../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_orf_platemap.txt",
    sep="\t",
).assign(Plate_Map_Name="JUMP-Target-1_orf_platemap")

# Read external metadata

compound_external_metadata = pd.read_csv(
    "../metadata/external_metadata/JUMP-Target-1_compound_metadata.tsv",
    sep="\t",
    usecols=["broad_sample", "pert_iname", "control_type"],
).query(
    'pert_iname == "DMSO" or pert_iname=="PFI-1" or pert_iname=="TG-101348" or pert_iname=="BI-2536"'
)

crispr_external_metadata = pd.read_csv(
    "../metadata/external_metadata/JUMP-Target-1_crispr_metadata.tsv",
    sep="\t",
    usecols=["broad_sample", "gene", "control_type"],
).query('gene == "BRD4" or gene == "PLK1" or control_type=="negcon"')

orf_external_metadata = pd.read_csv(
    "../metadata/external_metadata/JUMP-Target-1_orf_metadata.tsv",
    sep="\t",
    usecols=["broad_sample", "gene", "control_type"],
).query('gene == "BRD4" or control_type=="negcon"')

# Select one random CRISPR control treatment

crispr_external_metadata = pd.concat(
    [
        crispr_external_metadata.query('control_type!="negcon"'),
        crispr_external_metadata.query('control_type=="negcon"').sample(
            1, random_state=12527
        ),
    ],
    ignore_index=True,
)

# Select one random ORF control treatment

orf_external_metadata = pd.concat(
    [
        orf_external_metadata.query('control_type!="negcon"'),
        orf_external_metadata.query('control_type=="negcon"').sample(
            1, random_state=12527
        ),
    ],
    ignore_index=True,
)

# Merge platemaps and external metadata

compound_platemap_external_metadata = compound_platemap.merge(
    compound_external_metadata, on="broad_sample"
)

crispr_platemap_external_metadata = crispr_platemap.merge(
    crispr_external_metadata, on="broad_sample"
)

orf_platemap_external_metadata = orf_platemap.merge(
    orf_external_metadata, on="broad_sample"
)

# Choose one well for each compound perturbation

compound_platemap_external_metadata = (
    compound_platemap_external_metadata.groupby("pert_iname")
    .apply(lambda x: x.sample(1, random_state=12527))
    .reset_index(drop=True)
)

# Choose one well for CRISPR negcon

crispr_platemap_external_metadata = pd.concat(
    [
        crispr_platemap_external_metadata.query('control_type!="negcon"'),
        crispr_platemap_external_metadata.query('control_type=="negcon"').sample(
            1, random_state=12527
        ),
    ],
    ignore_index=True,
)

# Choose one well for ORF negcon

orf_platemap_external_metadata = pd.concat(
    [
        orf_platemap_external_metadata.query('control_type!="negcon"'),
        orf_platemap_external_metadata.query('control_type=="negcon"').sample(
            1, random_state=12527
        ),
    ],
    ignore_index=True,
)

# Merge compound platemap and external metadata with experiment metadata

compound_experiment = experiment_df.merge(
    compound_platemap_external_metadata, on="Plate_Map_Name", how="inner"
)

# Merge crispr platemap and external metadata with experiment metadata

crispr_experiment = experiment_df.merge(
    crispr_platemap_external_metadata, on="Plate_Map_Name", how="inner"
).fillna("crispr_control")

# Merge orf platemap and external metadata with experiment metadata

orf_experiment = experiment_df.merge(
    orf_platemap_external_metadata, on="Plate_Map_Name", how="inner"
)

# Choose one well for each compound perturbation per cell type

compound_experiment = (
    compound_experiment.groupby(["Cell_type", "pert_iname"])
    .apply(lambda x: x.sample(1, random_state=42))
    .reset_index(drop=True)
)

# Choose one well for CRISPR perturbation per cell type

crispr_experiment = (
    crispr_experiment.groupby(["Cell_type", "gene"])
    .apply(lambda x: x.sample(1, random_state=42))
    .reset_index(drop=True)
)

# Choose one well for ORF perturbation per cell type

orf_experiment = (
    orf_experiment.groupby(["Cell_type", "gene"])
    .apply(lambda x: x.sample(1, random_state=42))
    .reset_index(drop=True)
)

# Combine compound and crispr experiments

all_experiments = pd.concat(
    [compound_experiment, crispr_experiment, orf_experiment], join="outer", ignore_index=True
)

Process folder name

In [4]:
# Run aws s3 ls s3://cellpainting-gallery/cpg0000-jump-pilot/source_4/images/2020_11_04_CPJUMP1/images/ > output/folder_names.txt

folder_name_map = (
    (
        pd.read_csv(
            "output/folder_names.txt", sep="\t", header=None, names=["folder_name"]
        )
    )
    .assign(folder_name=lambda x: x.apply(lambda y: y.folder_name.strip()[4:-1], axis=1))
    .assign(
        Assay_Plate_Barcode=lambda x: x.apply(
            lambda y: y.folder_name.strip().split("__")[0], axis=1
        )
    )
)


Download images from S3

In [5]:
if not os.path.isdir('figures/BRD4/'):
    os.mkdir('figures/BRD4/')
if not os.path.isdir('figures/BRD4/U2OS/'):
    os.mkdir('figures/BRD4/U2OS/')
if not os.path.isdir('figures/BRD4/A549/'):
    os.mkdir('figures/BRD4/A549/')
if not os.path.isdir('figures/BRD4/U2OS/compound/'):
    os.mkdir('figures/BRD4/U2OS/compound')
if not os.path.isdir('figures/BRD4/A549/compound/'):
    os.mkdir('figures/BRD4/A549/compound')
if not os.path.isdir('figures/BRD4/U2OS/crispr/'):
    os.mkdir('figures/BRD4/U2OS/crispr')
if not os.path.isdir('figures/BRD4/A549/crispr/'):
    os.mkdir('figures/BRD4/A549/crispr')
if not os.path.isdir('figures/BRD4/U2OS/orf/'):
    os.mkdir('figures/BRD4/U2OS/orf')
if not os.path.isdir('figures/BRD4/A549/orf/'):
    os.mkdir('figures/BRD4/A549/orf')

download_images(all_experiments, folder_name_map)

100%|██████████| 18/18 [00:00<00:00, 1209.49it/s]


Create composite image

In [6]:
for row in tqdm(all_experiments.itertuples(), total=all_experiments.shape[0]):
    cell_type = row.Cell_type
    perturbation = row.Perturbation
    if perturbation == "compound":
        output = row.pert_iname
    else:
        output = row.gene

    images = []

    if not os.path.isfile(f"figures/BRD4/{cell_type}/{perturbation}/{output}/{output}_composite.png"):
        for filename in glob.glob(f"figures/BRD4/{cell_type}/{perturbation}/{output}/*"):
            img = tifffile.imread(filename)
            images.append(img)

        (width, height) = images[0].shape

        composite_image = np.asarray(
            [[0 for _ in range(width)] for _ in range(height)], dtype="uint16"
        )

        for i in range(height):
            for j in range(width):
                for k in range(len(images)):
                    composite_image[i][j] = int(
                        np.max([images[k][i][j] for k in range(len(images))])
                    )

        tifffile.imwrite(f"figures/BRD4/{cell_type}/{perturbation}/{output}/{output}_composite.png", composite_image)

100%|██████████| 18/18 [00:00<00:00, 70295.60it/s]


Create montage

In [7]:
treatment_dict = {
    "A549": {
        "compound": ["BI-2536", "PFI-1", "TG-101348", "DMSO"],
        "crispr": ["BRD4", "PLK1", "crispr_control"],
        "orf": ["BRD4", "RB1"],
    },
    "U2OS": {
        "compound": ["BI-2536", "PFI-1", "TG-101348", "DMSO"],
        "crispr": ["BRD4", "PLK1", "crispr_control"],
        "orf": ["BRD4", "RB1"],
    },
}


for cell_type in treatment_dict:
    images = []
    if not os.path.isfile(f"figures/BRD4/{cell_type}/{cell_type}_montage.png"):
        for perturbation_type in treatment_dict[cell_type]:
            for perturbation in treatment_dict[cell_type][perturbation_type]:
                filename = (
                    f"figures/BRD4/{cell_type}/{perturbation_type}/{perturbation}/{perturbation}_composite.png"
                )
                img = skimage.io.imread(filename)

                images.append(img)

        m = skimage.util.montage(
            images,
            grid_shape=(3, 3),
        )

        skimage.io.imsave(f"figures/BRD4/{cell_type}/{cell_type}_montage.png", m)